In [1]:
import pandas as pd
import requests
import json
import datetime
import csv

In [15]:
def get_pushshift_data(after, before, sub):

  #url = 'https://api.pushshift.io/reddit/search/submission/?q=science'
  #url = 'https://api.pushshift.io/reddit/search/submission/?&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)+'&sort=asc&sort_type=created_utc'
  url = 'https://api.pushshift.io/reddit/search/comment/?q=trump&after=7d&aggs=created_utc&frequency=hour&size=0'

  print(url)
  r = requests.get(url)
  data = json.loads(r.text, strict=False)
  return data['data']


def collect_subData(subm):
    subData = list()
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"
    try:
        body = subm['selftext']
    except KeyError:
        body = ''
    author = subm['author']
    subId = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc'])
    numComms = subm['num_comments']
    permalink = subm['permalink']

    subData.append((subId,title,body,url,author,score,created,numComms,permalink,flair))
    subStats[subId] = subData


def update_subFile():
    upload_count = 0
    location = ""
    print("input filename of submission file, please add .csv")
    filename = input()
    file = location + filename
    with open(file, 'w', newline='', encoding='utf-8') as file:
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Title","Body","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1

        print(str(upload_count) + " submissions have been uploaded into a csv file")


subStats = {}
subCount = 0
sub='nfl'
before = "1648789261"
after = "1651262461"


data = get_pushshift_data(after, before, sub)


while len(data) > 0:
    for submission in data:
        collect_subData(submission)
        subCount+=1
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = get_pushshift_data(after, before, sub)


print(len(data))


update_subFile()

https://api.pushshift.io/reddit/search/comment/?q=trump&after=7d&aggs=created_utc&frequency=hour&size=0


JSONDecodeError: ignored